
# 1. <U>Problem Statement</U>


An E Commerce company or DTH (you can choose either of these two domains) provider is facing a lot of competition in the current market and it has become a challenge to retain the existing customers in the current situation. Hence, the company wants to develop a model through which they can do churn prediction of the accounts and provide segmented offers to the potential churners. In this company, account churn is a major thing because 1 account can have multiple customers. hence by losing one account the company might be losing more than one customer.

You have been assigned to develop a churn prediction model for this company and provide business recommendations on the campaign.

Your campaign suggestion should be unique and be very clear on the campaign offer because your recommendation will go through the revenue assurance team. If they find that you are giving a lot of free (or subsidized) stuff thereby making a loss to the company; they are not going to approve your recommendation.

Hence be very careful while providing campaign recommendation.



# 2. <U>Data Collection</U>

The Dataset is mentioned as below:

Customer Churn Data.xlsx

Dataset granted by: PARIKSHITH A


# 3. <U>Data Ingestion</U>


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cf=pd.read_excel('Customer Churn Data.xlsx',sheet_name='Data for DSBA') # reading the data set

In [ ]:
cf.head() #top 5 rows get displayed

In [ ]:
cf.tail() #last bottom 5 rows get displayed


# 4. <U>Data Cleaning </U>


Checking for null values and duplicate values

In [ ]:
cf.dtypes # data types of each individual column get printed

Here, there are 12 categorical characteristics in this list, 5 of which are float types and the remaining 7 are all integer types.

In [ ]:
cf.isnull().sum() # checking each individual column that there are Null values or not|

In this case, we'll replace the column with NULL values with the median and mode, i.e., categorical by mode and numerical by mean.

In [ ]:
cf.duplicated().sum() #there no duplicated rows

We are in the process of discarding the columns which satisfy neither our purposes nor those of our target.

In [ ]:
cf.drop(['AccountID','Payment','Gender','Marital_Status','cashback','Login_device'],axis=1,inplace=True) #dropping the unwanted column

In [ ]:
cf.drop(['coupon_used_for_payment'],axis=1,inplace=True)

In [ ]:
cf.dtypes # data types of each individual column get printed

In [ ]:
# Identify object columns except for 'Column1'
object_columns = cf.select_dtypes(include=['object']).columns.drop('account_segment')


# Convert object columns to float using pd.to_numeric()
cf[object_columns] = cf[object_columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
cf['Churn'] = cf['Churn'].astype(float) #converting int to float

In [ ]:
cf.dtypes

Let us now substitute mean and mode for the NULL values.


In [ ]:
# cf = cf.fillna(cf.median()) #this is for numerical column

numeric_columns = cf.select_dtypes(include=['float', 'int']).columns
cf[numeric_columns] = cf[numeric_columns].fillna(cf[numeric_columns].median())

In [ ]:
#now lets check again that there is null counts or not
cf.isnull().sum()

In [ ]:
#now lets fill object column null values with mode
object_columns = cf.select_dtypes(include=['object']).columns
cf[object_columns] = cf[object_columns].fillna(cf[object_columns].mode().iloc[0])

In [ ]:
#now lets check again that there is null counts or not
cf.isnull().sum()

In [ ]:
# Check unique values in each column
for column in cf.columns:
    unique_values = cf[column].unique()
    print(f"Unique values in {column}: {unique_values}")


# 5. <U>Exploratory Data Analysis (EDA) </U>


In [ ]:
cf.sample(6) #this line will randomly select any five column from dataset

> #### <em>Attributes Information:</em>
>- churn: account churn flag (Target)
>- Tenure:Tenure of account
>- City_Tier:Tier of primary customer's city
>- cc_contacted_Ly:How many time the customer of the account has contacted customer care in last 12months
>- Service_Score:Satisfaction score given by customers of the account on service provided by company
>- Account_user_count:Number of customers tagged with this account
>- account_segment:Account segmentation on the basis of spend
>- CC_Agent_Score:Satisfaction score given by customers of the account on customer care service provided by company
>- rev_per_month:Monthly average revenue generated by account in last 12 months
>- complain_ly:Any complaints has been raised by account in last 12 months
>- rev_growth_yoy:revenue growth percentage of the account (last 12 months vs last 24 to 13month)
>- Day_Since_CC_connect:Number of days since no customers in the account has contacted the customer care
>
> .


In [ ]:
cf.shape #{checking the number of rows and column}

In [ ]:
cf.info() #{ checking the dataypes and count of null values if present}

In [ ]:
cf['Churn'].value_counts() # Here it can be seen that the number of 1's is very less as compared to number of 0's. So that dataset is imbalanced dataset.

It can be seen that the number of 1s is much lower than the number of 0s. As a result, that dataset is imbalanced.

### <em> Separating Numerical and Categorical Features </em>

In [ ]:
num_feature= [fea for fea in cf.columns if cf[fea].dtype !=object]
cat_feature= [fea for fea in cf.columns if cf[fea].dtype==object]

In [ ]:
print("We have {} Numerical features : {}".format(len(num_feature),num_feature))
print()
print("We have {} Categorical features : {}".format(len(cat_feature),cat_feature))

#### Statistical Description

In [ ]:
cf.describe().T

#### Univariate Analysis

In [ ]:
plt.figure(figsize=(15,17))
plt.suptitle('Univariate Analysis',fontsize=20,fontweight='bold',y=1)

for i in range(0,len(num_feature)):
    plt.subplot(5,3,i+1)
    sns.kdeplot(x=cf[num_feature[i]],shade=True,color='g')
    plt.xlabel(num_feature[i])
    plt.tight_layout()

>Observations
>- CC_Agent_scored are normally distributed.
>- Tenure,Day_Since_CC_connect,rev_per_month,CC_Contacted_ly, is havily left skewed.
>- Almost 50% of the customefr are not active.

In [ ]:
plt.figure(figsize=(15,17)) #fixing the size of graph
plt.suptitle('Scatter plot for Numerical features',fontsize=20,fontweight='bold',y=1) # placing at top of the graph heading.

for i in range(0,len(num_feature)):
    plt.subplot(5,3,i+1)
    sns.scatterplot(y=num_feature[i],x=cf.index,data=cf,color='b',hue='Churn')
    plt.legend(loc="upper right")
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(20,40))
plt.suptitle('Count plot for Categorical features',fontsize=20,fontweight='bold',y=1)

for i in range(0,len(cat_feature)):
    plt.subplot(8,2,i+1)
    sns.countplot(y=cat_feature[i],data=cf)
    plt.tight_layout()

#### Bivariate Analysis

In [ ]:
plt.figure(figsize=(10,20))
sns.pairplot(cf)
plt.show()

#### Bivariate Analysis w.r.t Target Variable

In [ ]:
plt.figure(figsize=(15,17))
plt.suptitle('Bivariate Analysis w.r.t Target Variable',fontsize=20,fontweight='bold',y=1)

for i in range(0,len(num_feature)):
    plt.subplot(5,3,i+1)
    sns.barplot(y=num_feature[i],x='Churn',data=cf,ci=0)
    plt.tight_layout()

In [ ]:
# Select only numeric columns
numeric_columns = cf.select_dtypes(include=['float', 'int'])

# Calculate the correlation matrix
corr_matrix = numeric_columns.corr()

# Plot the correlation matrix using a heatmap
plt.figure(figsize=(15, 14))
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.histplot(x='Day_Since_CC_connect',hue='Churn',data=cf)
plt.show()

In [ ]:
sns.countplot(x=cf.Churn,hue=cf.City_Tier,color='red')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(x='Churn',hue='Service_Score',data=cf,color='red')

In [ ]:
plt.figure(figsize=(15,30))
plt.suptitle('BOX PLOT for all numerical feature',fontsize=20,fontweight='bold',y=1)

for i in range(0,len(num_feature)):
    plt.subplot(8,2,i+1)
    sns.boxplot(data=cf[num_feature],orient='h') # checking the outliers are present or not
    plt.tight_layout()


# 6. <U>Data Preprocessing</U>


Feature encoding to convert the categorical features into numerical values

In [ ]:
cf.head() #checking top 5 rows of the data set

In [ ]:
#Now doing one_hot_encoding to convert categorical features into numerical feature
cf1=pd.get_dummies(data=cf,columns=['account_segment'],drop_first=True)

In [ ]:
cf1.head()

### Extracting the target column into separate vectors for training set and test set

In [ ]:
X=cf1.drop(['Churn'],axis=1) #dropping the label
Y=cf1['Churn']
X.shape,Y.shape #checking shape

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
!pip install imbalanced-learn

### Balancing the dataset by using oversampling technique

In [ ]:
from imblearn.combine import SMOTETomek
import matplotlib.pyplot as plt

smk = SMOTETomek()
X_res, Y_res = smk.fit_resample(X, Y)
%matplotlib inline

In [ ]:
X_res.shape,Y_res.shape #after balancing the data set checking new shape

## <em> Splitting the dataset into train and test data </em>

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_res,Y_res,test_size=0.2,random_state=0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

In [ ]:
Y_test.value_counts(normalize=True)*100

In [ ]:
Y_train.value_counts(normalize=True)*100

In [ ]:
model=LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)
Y_predict=model.predict(X_test)
type(model)

In [ ]:
model_score=model.score(X_test,Y_test)
print('Accuracy score is',model_score*100)

In [ ]:
confusion_mat = confusion_matrix(Y_test, Y_predict) # Calculate the confusion matrix

print("Confusion Matrix:\n", confusion_mat)  # Print the confusion matrix


## <em> Buiding the Model </em>

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import matplotlib.style
plt.style.use('classic')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
import statsmodels.formula.api as smf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn import metrics

In [ ]:
param_grid = {
'criterion': ['gini'],
'max_depth': [10],#[10,20,30,50]#[3,5,7,9]
'min_samples_leaf': [250],#[100,150,200,250]# 1-3% 50-150
'min_samples_split': [750]#[150,300,450,600,750] # 150 - 450
}
dtcl = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtcl, param_grid = param_grid, cv = 10)

In [ ]:
grid_search.fit(X_train, Y_train)
print(grid_search.best_params_)
best_grid = grid_search.best_estimator_
best_grid
#{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 250, 'min_samples_split': 750}

In [ ]:
ytrain_predict = best_grid.predict(X_train)
ytest_predict = best_grid.predict(X_test)

## <em>Predicting on Training and Test dataset</em>

In [ ]:
ytest_predict
ytest_predict_prob=best_grid.predict_proba(X_test)
ytest_predict_prob
pd.DataFrame(ytest_predict_prob).head()

## <em>Model Evaluation</em>

### Confusion Matrix for the Training Data

In [ ]:
confusion_matrix(Y_train, ytrain_predict)

In [ ]:
#Train Data Accuracy
cart_train_acc=best_grid.score(X_train,Y_train)
cart_train_acc

In [ ]:
print(classification_report(Y_train, ytrain_predict))

In [ ]:
cart_metrics=classification_report(Y_train, ytrain_predict,output_dict=True)
cf1=pd.DataFrame(cart_metrics).transpose()
cart_train_precision=round(cf1.loc["1.0"][0],2)
cart_train_recall=round(cf1.loc["1.0"][1],2)
cart_train_f1=round(cf1.loc["1.0"][2],2)
print ('cart_train_precision ',cart_train_precision)
print ('cart_train_recall ',cart_train_recall)
print ('cart_train_f1 ',cart_train_f1)

### Confusion Matrix for the Testing Data

In [ ]:
confusion_matrix(Y_test, ytest_predict)

In [ ]:
#Test Data Accuracy
cart_test_acc=best_grid.score(X_test,Y_test)
cart_test_acc

In [ ]:
print(classification_report(Y_test, ytest_predict))

In [ ]:
cart_metrics=classification_report(Y_test, ytest_predict,output_dict=True)
cf1=pd.DataFrame(cart_metrics).transpose()
cart_test_precision=round(cf1.loc["1.0"][0],2)
cart_test_recall=round(cf1.loc["1.0"][1],2)
cart_test_f1=round(cf1.loc["1.0"][2],2)
print ('cart_train_precision ',cart_test_precision)
print ('cart_train_recall ',cart_test_recall)
print ('cart_train_f1 ',cart_test_f1)

## <em>Building a Random Forest Classifier</em>

Grid Search for finding out the optimal values for the hyper parameters

Due to large volume of data, trying for different parameter values in the gridsearch with higher cv value will lead
to performance issues and model will run for much longer time

In [ ]:
param_grid = {#put a grid for hyperparameters
'max_depth': [8,9],
'max_features': [8,9],#[5,4,6],
'min_samples_leaf': [250,150],
'min_samples_split': [750,500],
'n_estimators': [100,150]#150,250
}
rfcl = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rfcl, param_grid = param_grid, cv = 3)

In [ ]:
grid_search.fit(X_train, Y_train)

In [ ]:
param_grid

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
best_grid

## <em>Predicting the Training and Testing data </em>

In [ ]:
ytrain_predict = best_grid.predict(X_train)
ytest_predict = best_grid.predict(X_test)

## <em>RF Model Perfotrmance Evaluation on Training data</em>

In [ ]:
confusion_matrix(Y_train,ytrain_predict)

In [ ]:
rf_train_acc=best_grid.score(X_train,Y_train)
rf_train_acc
rf_test_acc=best_grid.score(X_test,Y_test)
rf_test_acc

In [ ]:
print(classification_report(Y_train,ytrain_predict))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
rf_metrics=classification_report(Y_train, ytrain_predict,output_dict=True)
cf1=pd.DataFrame(rf_metrics).transpose()
rf_train_precision=round(cf1.loc["1.0"][0],2)
rf_train_recall=round(cf1.loc["1.0"][1],2)
rf_train_f1=round(cf1.loc["1.0"][2],2)
rf_test_recall=round(cf1.loc["1.0"][1],2)
print ('rf_train_precision ',rf_train_precision)
print ('rf_train_recall ',rf_train_recall)
print ('rf_train_f1 ',rf_train_f1)

In [ ]:
rf_train_fpr, rf_train_tpr,_=roc_curve(Y_train,best_grid.predict_proba(X_train)[:,1])
plt.plot(rf_train_fpr,rf_train_tpr,color='green')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
rf_train_auc=roc_auc_score(Y_train,best_grid.predict(X_train))
print('Area under Curve is', rf_train_auc)

## <em>Building a Neural Network Classifier</em>

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [ ]:
param_grid = {
'hidden_layer_sizes': [64,128],#[32,64,128], #Multiple layers(200,120)
'max_iter': [100,200,300],
'solver': ['adam','sgd'], #sgd
}
nncl = MLPClassifier(tol = 0.01)
grid_search = GridSearchCV(estimator = nncl, param_grid = param_grid, cv = 3)

In [ ]:
grid_search.fit(X_train_scaled, Y_train)
#{'hidden_layer_sizes': 32, 'max_iter': 200, 'solver': 'adam', 'tol': 0.01}

In [ ]:
best_grid = grid_search.best_estimator_
best_grid

## <em>Predicting the Training and Testing data</em>

In [ ]:
ytrain_predict = best_grid.predict(X_train_scaled)
ytest_predict = best_grid.predict(X_test_scaled)

## <em> NN Model Performance Evaluation on Training data

In [ ]:
confusion_matrix(Y_train,ytrain_predict)

In [ ]:
nn_train_acc=best_grid.score(X_train_scaled,Y_train)
nn_train_acc

In [ ]:
print(classification_report(Y_train,ytrain_predict))

In [ ]:
nn_metrics=classification_report(Y_train, ytrain_predict,output_dict=True)
cf1=pd.DataFrame(nn_metrics).transpose()
nn_train_precision=round(cf1.loc["1.0"][0],2)
nn_train_recall=round(cf1.loc["1.0"][1],2)
nn_train_f1=round(cf1.loc["1.0"][2],2)
print ('nn_train_precision ',nn_train_precision)
print ('nn_train_recall ',nn_train_recall)
print ('nn_train_f1 ',nn_train_f1)

In [ ]:
nn_train_fpr, nn_train_tpr,_=roc_curve(Y_train,best_grid.predict_proba(X_train)[:,1])
plt.plot(nn_train_fpr,nn_train_tpr,color='black')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
nn_train_auc=roc_auc_score(Y_train,best_grid.predict(X_train))
print('Area under Curve is', nn_train_auc)

## <em>NN Model Performance Evaluation on Test data

In [ ]:
confusion_matrix(Y_test,ytest_predict)

In [ ]:
nn_test_acc=best_grid.score(X_test_scaled,Y_test)
nn_test_acc

In [ ]:
print(classification_report(Y_test,ytest_predict))

In [ ]:
nn_metrics=classification_report(Y_test, ytest_predict,output_dict=True)
cf1=pd.DataFrame(nn_metrics).transpose()
nn_test_precision=round(cf1.loc["1.0"][0],2)
nn_test_recall=round(cf1.loc["1.0"][1],2)
nn_test_f1=round(cf1.loc["1.0"][2],2)
print ('nn_test_precision ',nn_test_precision)
print ('nn_test_recall ',nn_test_recall)
print ('nn_test_f1 ',nn_test_f1)

In [ ]:
nn_test_fpr, nn_test_tpr,_=roc_curve(Y_test,best_grid.predict_proba(X_test)[:,1])
plt.plot(nn_test_fpr,nn_test_tpr,color='black')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
nn_test_auc=roc_auc_score(Y_test,best_grid.predict(X_test))
print('Area under Curve is', nn_test_auc)

In [ ]:
best_grid.score

In [ ]:
index=['Accuracy','Recall','Precision','f1_Score']
data=pd.DataFrame({'CART Train':[cart_train_acc,cart_train_recall,cart_train_precision,cart_train_f1],
                    'CART Test':[cart_test_acc,cart_test_recall,cart_test_precision,cart_test_f1],
                    'Random forest train':[rf_train_acc,rf_train_recall,rf_train_precision,rf_train_f1],
                    'Neural Network train':[nn_train_acc,nn_train_recall,nn_train_precision,nn_train_f1],
                    'Neural Network test':[nn_test_acc,nn_test_recall,nn_test_precision,nn_test_f1]},index=index)
round(data,6)

### <em>Based on the provided output from the dataset, here is an interpretation of the accuracy scores for different models:</em>

####  1. Decision Tree (CART) Model:
- Train Data Accuracy: The decision tree model achieved a training accuracy of approximately 84.82% (0.8482). This indicates that the model correctly predicted the churn outcome for about 84.82% of the training data.
- Test Data Accuracy: The neural network model achieved a test accuracy of approximately 84.3% (0.8430). This suggests that the model correctly predicted the churn outcome for approximately 84.3% of the test data.

#### 2. Random Forest Model:
- Train Data Accuracy: The random forest model achieved a training accuracy of around 85.42% (0.8542). This suggests that the model correctly predicted the churn outcome for approximately 85.42% of the training data.

#### 3. Neutral Network Model:
- Train Data Accuracy: The neural network model achieved a training accuracy of approximately 88.24% (0.8824). This indicates that the model correctly predicted the churn outcome for around 88.24% of the training data.
- Test Data Accuracy: The neural network model achieved a test accuracy of approximately 88.25% (0.8825). This suggests that the model correctly predicted the churn outcome for approximately 82.25% of the test data.

### <em>Based on above observation our data set is good fit and good model</em>
### <em>Based on these results, the neural network model seems to have the highest accuracy both on the training and test data, indicating its better performance compared to the decision tree and random forest models. However, it is important to consider other factors such as model complexity, interpretability, and potential overfitting when selecting the best model for your specific churnprediction task.</em>